**This tutorial uses a dataset of human height and weight to introduce the following:**

* Numpy and Pyplot, the basic tools for any Python data science or machine learning project
* Tensor indexing, broadcasting, and normalization: common data manipulation techniques

**After you've read, run, and understood the code, try to modify it as follows:**
* Easy: Visualize only the data points 10...20 
* Harder: Visualize how the mean of height changes when you compute it from ranges of different lengths. You might make use of a for loop (Google python for loop if you don't know how). It is important to understand that mean and standard deviation computed from random data are also random variables; using them as estimates of the actual mean and sd becomes more reliable with more data.

Model solutions are provided in the same folder, but first try to solve the problem yourself, at least for a few minutes. *Think of this as a puzzle game with optional hints that you will check only if you have to.*

Let's start by importing the tools we need:

In [ ]:
#the pylab inline may be needed to make figures visible in Jupyter, depending on your Anaconda installation
%pylab inline
import numpy as np
import matplotlib.pyplot as pp
import pandas as pd

Now, let's load some data to process. We'll use the human height and weight dataset from Kaggle, which is in .csv format. This is a common and perhaps the most simple data format -- try opening it in a text editor. Loading .csv using Numpy is easy.

Note that we skip one row which has the headers, and we only extract the second and third column. In Numpy, indices start from zero, i.e., 1 means the second instead of the first column.

Let's load some data. For comma separated values, Pandas provides the best tools, including the ability to directly read from an URL to a data frame (a kind of database object). When running this notebook in Colab, this avoids you having to upload the file manually, or mount your Google Drive.

In [ ]:
dataframe=pd.read_csv("https://raw.githubusercontent.com/PerttuHamalainen/MediaAI/master/Code/Datasets/weight-height.csv")
print(dataframe)

To be able to do math on the numbers, we just convert them to a Numpy array. We only use the "Height" and "Weight" columns. The ```[:,1:]``` is a bit of array indexing magic you will be using often. Typically, the first array index denotes rows and the second denotes columns. The colon ```:``` is an operator for defining a range of indices. ```a:b``` means one takes elements a...b-1, ```a:``` means elements starting from a, and ```:a``` means all elements up to a-1. ```:``` alone means all elements. Indexing starts at 0. Thus, ```[:,1:]``` means we take all rows, and everything but the first column. 

**Now re-read the above until ```[:,1:]``` is your friend and no longer scary.**

In [ ]:
data=np.array(dataframe)
data=data[:,1:]
print(data)

One last data conversion trick: Enforce that the data is floating point values. Sometimes the Pandas to Numpy data conversion doesn't parse the values correctly, which is why it's good to add this. Note: You can comment the line out using "#" and try running the notebook without it.

In [ ]:
data=data.astype(np.float) 

When printing the data, we see that it is a 2D-array or 'tensor'. Machine learning typically utilizes tensors to store and manipulate data. Numpy is the basic package for manipulating tensors on the CPU, and Tensorflow and PyTorch allow one to do the same on the GPU, with some additional features.

** IMPORTANT: Becoming fluent in thinking and coding with tensors takes time but is extremely vital.** 

Let's investigate the data a bit more.

For example, each tensor has a shape: an 1D array that tells about the tensor dimensions. We can also query the number of dimensions with ndim(). The number of dimensions is also called the "rank" of the tensor. In linear algebra terms, a scalar is a rank-0 tensor, a vector is a rank-1 tensor, and a matrix is a rank-2 tensor.

Unlike with vectors and matrices, A\*B multiplies the tensors A and B elementwise, and if A and B are vectors, one can use np.dot(A,B) to compute the dot product.

In [ ]:
print("Data shape ",data.shape,", rank ",np.ndim(data))

In other words, there's 10000 data points, each with 2 values. The tensor is a 2D one. The first dimension indexes over data points, and the second over different variables. In this dataset, the first variable is height and the second is weight.

As already explained above, Numpy allows very flexible indexing of tensors. As an example, let's do some unit conversion: The height values are in inches, and we can multiply them with 2.54 to convert them to meters:

In [ ]:
data[:,0]*=2.54  #Here, we use the *= shorthand, which is the same as data[:,0]=data[:,0]*2.54
print(data)

Note: if you run the code above for many times, you keep multiplying the height, as Jupyter maintains the program state. You can fix this by 1) manually re-running the data loading code above or 2) selecting "Restart & Run All" from the Kernel menu at the top of the page.

Let's now do similar scaling for weight, which is given in pounds.

In [ ]:
data[:,1]*=0.45359237
print(data)

Now, let's try some data visualization using PyPlot. 

**Note: if you run this code outside Jupyter, you first need to create a new figure using pp.figure() and finally say pp.show() to make the figure visible.**

In [ ]:
pp.scatter(data[:,0],data[:,1],marker=".")
pp.title("Relation of height and weight")
pp.xlabel("Height (centimeters)")
pp.ylabel("Weight (kilograms)")

Not surprisingly, there's a strong correlation between height and weight. We can see that a 2m tall person typically weights around 110 kg. 

In the next exercise, we'll use some machine learning models to predict weight from height and vice versa, but let's now continue a bit more on introducing basic Numpy and Pyplot features.

Above, we've already used the ":" symbol to denote "all indices". It can also be used for defining arbitrary ranges. Note the following:
* The indexing is zero-based, i.e., index 1 is the second element and index 0 is the first.
* A range a:b includes a but not b. Instead, the last index in the range is b-1. 

Thus, this is how you print three elements starting from the second one:

In [ ]:
print(data[1:4])

Note that giving only one index prints out the rest of the dimensions: A 2D tensor is effectively an array of 1D tensors. Equivalently, you can use:

In [ ]:
print(data[1:4,:])

Finally, let's introduce a few more common tools and tricks: Compute the mean and standard deviation of the data.

In [ ]:
#Numpy provides utilities for mean and standard deviation:
m=np.mean(data,axis=0) #we compute the mean across the first axis, i.e., the data entries
sd=np.std(data,axis=0)
print(m,sd)

The results show that human average height is 169 cm, with standard deviation of 10cm. Now, let's normalize the data: subtract mean and divide by standard deviation. This is a common operation to make data *zero-mean, unit standard deviation*, which is what neural networks often need to work efficiently.

**Important note: the normalization below makes use of so-called broadcasting**, where numpy automatically expands tensors of lower rank to match higher-rank tensors. The data is a 2D tensor and m and sd are 1D tensors, as printed above. Broadcasting means that the 1D tensor is "replicated" along a new dimension, making it a 2D tensor. This is a generalization of what happens when you multiply a tensor with a plain number, i.e., a scalar or rank-0 tensor.

Broadcasting also works for tensors of similar rank in cases where one tensor has only 1 element along some dimension, i.e., a shape [1,3] tensor can be multiplied with shape [4,3] tensor.   

In [ ]:
normalized=(data-m)/sd
print(normalized)
pp.scatter(normalized[:,0],normalized[:,1])

Typical to normally distributed (Gaussian) data, we see that values beyond 3 standard deviations from the mean are extremely rare.

If you want to know how rare a specimen you are, you can try the following:

In [ ]:
myHeight=178
print("My height is",myHeight,"cm, which is ",(myHeight-m[0])/sd[0],"standard deviations from the mean")

Finally, to transform back from normalized data to actual values, you can use:

In [ ]:
denormalized=normalized*sd+m
print(denormalized)

For pedagogical purposes, we showed above how to manually compute the mean and std and do the normalization/denormalization. Again, this is common because most neural networks work best when fed data with zero mean and standard deviation of one.

**In practice, one typically uses a scaler utility.** For example, the sklearn package provides one:

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(data)
normalized=scaler.transform(data)
denormalized=scaler.inverse_transform(normalized)
pp.scatter(normalized[:,0],normalized[:,1],label="normalized")
pp.scatter(denormalized[:,0],denormalized[:,1],label="denormalized")
pp.legend()

This concludes the tutorial. We also learned how to plot multiple datasets in the same figure with legends, which will often be useful. 

**Now, go back to the top and try to implement the suggested modifications.**